In [3]:
# import packages
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:,.6f}'.format

### Load Covid Symptom Data and Prep 

In [4]:
# Load Data
positive_data = pd.read_csv('/data/project/ubrite/covid19-hackathon/Team4_staging_area/parsed_data/positive_condition_categorical.csv')
negative_data = pd.read_csv('/data/project/ubrite/covid19-hackathon/Team4_staging_area/parsed_data/negative_condition_categorical.csv')
all_data =  positive_data.append(negative_data, ignore_index=True)

all_data.head()

,Unnamed: 0,35211292,35207924,45534424,133835,35207977,134668,138384,35208543,35208542,...,37200942,45601843,45562105,45541003,45555454,45541002,35208428,4199522,37309673,45533619
0,0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,2,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,3,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
4,4,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [5]:
all_data = all_data.fillna(value=0)
all_data = all_data.drop(['Unnamed: 0'], axis=1)

In [6]:
positive_data.shape

(910, 9227)

In [7]:
negative_data.shape

(6339, 24059)

In [8]:
positive_data.shape[0]+negative_data.shape[0]

7249

In [9]:
# Shape
all_data.shape

(7249, 24961)

In [10]:
all_data['target']

0       1
1       1
2       1
3       1
4       1
       ..
7244    0
7245    0
7246    0
7247    0
7248    0
Name: target, Length: 7249, dtype: int64

### Begin Modeling <a name="beginModel"></a>

<a name="woe"></a>

<div class="alert alert-info">
<b>Transformations, Weight of Evidence, & Information Value</b>
</div>

#### Set up Weight of Evidence Transform
following https://github.com/Sundar0989/WOE-and-IV/blob/master/WOE_IV.ipynb

In [11]:
# binning function
max_bin = 20
force_bin = 0

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]    
    df2 = notmiss.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.sum().Y.index
    d3["MAX_VALUE"] = d3["MIN_VALUE"]
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.COUNT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.COUNT
    d3["DIST_EVENT"] = d3.EVENT/d3.sum().EVENT
    d3["DIST_NON_EVENT"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["IV"] = (d3.DIST_EVENT-d3.DIST_NON_EVENT)*np.log(d3.DIST_EVENT/d3.DIST_NON_EVENT)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'DIST_EVENT','DIST_NON_EVENT','WOE', 'IV']]      
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)

In [12]:
all_data.columns

Index(['35211292', '35207924', '45534424', '133835', '35207977', '134668',
       '138384', '35208543', '35208542', '35206859',
       ...
       '37200942', '45601843', '45562105', '45541003', '45555454', '45541002',
       '35208428', '4199522', '37309673', '45533619'],
      dtype='object', length=24961)

In [13]:
# remove severity columns
all_data_clean = all_data.drop(['person_id'], axis=1)

In [14]:
# Transform
final_symptom_iv, symptom_iv = data_vars(all_data_clean,all_data_clean.target)

/share/apps/rc/software/Anaconda3/2020.02/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [15]:
symptom_iv.to_csv('/data/project/ubrite/covid19-hackathon/Team4_staging_area/parsed_data/condition_code_iv.csv')

In [23]:
# preview Weights Table
final_symptom_iv

,VAR_NAME,MIN_VALUE,MAX_VALUE,COUNT,EVENT,EVENT_RATE,NONEVENT,NON_EVENT_RATE,DIST_EVENT,DIST_NON_EVENT,WOE,IV
0,J,0.000000,0.000000,2819,310,0.109968,2509,0.890032,0.344828,0.400479,-0.149616,0.013266
1,J,1.000000,1.000000,4345,589,0.135558,3756,0.864442,0.655172,0.599521,0.088767,0.013266
2,L,0.000000,0.000000,4772,612,0.128248,4160,0.871752,0.680756,0.664006,0.024913,0.001274
3,L,1.000000,1.000000,2392,287,0.119983,2105,0.880017,0.319244,0.335994,-0.051138,0.001274
4,B,0.000000,0.000000,4701,510,0.108488,4191,0.891512,0.567297,0.668955,-0.164833,0.043980
5,B,1.000000,1.000000,2463,389,0.157937,2074,0.842063,0.432703,0.331045,0.267796,0.043980
6,Z,0.000000,0.000000,2691,521,0.193608,2170,0.806392,0.579533,0.346369,0.514718,0.222880
7,Z,1.000000,1.000000,4473,378,0.084507,4095,0.915493,0.420467,0.653631,-0.441177,0.222880
8,N,0.000000,0.000000,2903,427,0.147089,2476,0.852911,0.474972,0.395211,0.183835,0.025943
9,N,1.000000,1.000000,4261,472,0.110772,3789,0.889228,0.525028,0.604789,-0.141428,0.025943


In [24]:
symptom_iv.sort_values('IV')

,VAR_NAME,IV
16,W,0.000114
18,Y,0.000770
8,L,0.001274
4,H,0.004835
17,X,0.005539
13,Q,0.007272
9,M,0.008410
15,V,0.008749
6,J,0.013266
12,P,0.023506
